In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

# Generar los DataFrames

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str, dtype_backend='numpy_nullable')
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [4]:
db_file2 = 'productos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

productos_sin_compra = pd.read_excel(db_file2_path_str, dtype_backend='numpy_nullable')

In [5]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [6]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [7]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [8]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [9]:
compras_odoo = pd.merge(
                    compras_linea,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

compras = pd.concat([compras_odoo, productos_sin_compra])

cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
compras[cols_to_Int64] = compras[cols_to_Int64].astype('Int64')

compras['tolerance_order_date'] = compras['order_date'].dt.normalize() - pd.Timedelta(days=5)

# Checks

In [10]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')


Todo correcto con check1


In [11]:
check_costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

check2 = check_costo_venta[check_costo_venta['product_price'].isna()][['product_id', 'product_name_x']]

print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 21 renglones sin costo de la venta.
Los productos sin costo son los siguientes:


,product_id,product_name_x
10061,14757,Nuevo *0*
10282,28586,Anticipo
13415,28638,Servicios de Facturación
18005,4,Anticipo (PdV)


# Primera ronda

### Sacar los productos con compras a varios proveedores

In [12]:
productos_varios_prov = (compras_odoo[['partner_id', 'partner_name', 'product_id_pp', 'product_name']]
                    .sort_values('partner_id').groupby('product_id_pp')
                    .agg({'product_name': ['first'], 'partner_id': ['mean', 'first']})
                    )

productos_varios_prov.columns = ['product_name', 'mean', 'first']
productos_varios_prov['diff'] = productos_varios_prov['mean']  == productos_varios_prov['first']

productos_varios_prov_ids = productos_varios_prov.loc[productos_varios_prov['diff'] == False].reset_index()['product_id_pp']

ventas_productos_varios_prov = ventas_año[ventas_año['product_id'].isin(productos_varios_prov_ids)]
compras_productos_varios_prov = compras[compras['product_id_pp'].isin(productos_varios_prov_ids)]

ventas_año_diff = ventas_año[~ventas_año['product_id'].isin(productos_varios_prov_ids)]

In [13]:
costo_venta = (
    pd.merge_asof(
        left = ventas_año.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        # tolerance = pd.Timedelta(days=1),
        
        direction = 'backward')
)

In [14]:
compras_prod_varios_prov['product_qty'] = compras_prod_varios_prov['product_qty'].astype('Float64')
compras_prod_varios_prov['vendedora'] = compras_prod_varios_prov['vendedora'].convert_dtypes()

NameError: name 'compras_prod_varios_prov' is not defined

In [ ]:
# Checar si es mejor primero (ronda 1 de concatenación), sacar de las compras los 212 'prod_varios_prov_ids', para que se quede el resto del DF en esa columna como NaN y en una
# segunda vuelta calificar por niveles el resto de los renglones NaN

v = pd.concat([ventas_año_diff, ventas_prod_varios_prov]).sort_values(by='invoice_date')

### 2da vuelta al costo de ventas

In [ ]:
compras_prod_varios_prov_odoo = compras_prod_varios_prov[~compras_prod_varios_prov['line_id'].isna()]

In [ ]:
# ventas_costo_venta_prod_varios_prov = (
#     pd.merge_asof(
#         left = ventas_prod_varios_prov.sort_values('invoice_date'),
#         right = compras_prod_varios_prov.sort_values('order_date'), 
        
#         left_by = ['product_id', 'salesperson_name', 'quantity'], 
#         right_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        
#         left_on = 'invoice_date', 
#         right_on = 'order_date', 

#         tolerance = pd.Timedelta(days=5),
        
#         direction = 'nearest')
# )

In [ ]:
compras_costo_venta_prod_varios_prov = (
    pd.merge_asof(
        left = compras_prod_varios_prov_odoo.sort_values('order_date'), 
        right = ventas_prod_varios_prov.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        tolerance = pd.Timedelta(days=5),
        
        direction = 'nearest')
)

ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (compras_costo_venta_prod_varios_prov['fact_line_id'].duplicated()), 'fact_line_id']
ids_order_line_con_ids_fact_line_repetidas = compras_costo_venta_prod_varios_prov.loc[compras_costo_venta_prod_varios_prov['fact_line_id'].isin(ids_fact_line_repetidas), 'line_id']
costo_ventas_2da_vuelta_merge1 = compras_costo_venta_prod_varios_prov.loc[(~compras_costo_venta_prod_varios_prov['fact_line_id'].isna()) & (~compras_costo_venta_prod_varios_prov['line_id'].isin(ids_order_line_con_ids_fact_line_repetidas)), ['fact_line_id', 'line_id']]

ventas_prod_varios_prov_after_merge1 = ventas_prod_varios_prov.loc[~ventas_prod_varios_prov['fact_line_id'].isin(costo_ventas_2da_vuelta_merge1['fact_line_id'])]
compras_prod_varios_prov_odoo_after_merge1 = compras_prod_varios_prov_odoo.loc[~compras_prod_varios_prov_odoo['line_id'].isin(costo_ventas_2da_vuelta_merge1['line_id'])]

In [ ]:
#Sólo para explicación, borrar
print(len(ventas_prod_varios_prov.loc[~ventas_prod_varios_prov['fact_line_id'].isin(costo_ventas_2da_vuelta_merge1['fact_line_id'])]))
print(len(compras_prod_varios_prov_odoo.loc[~compras_prod_varios_prov_odoo['line_id'].isin(costo_ventas_2da_vuelta_merge1['line_id'])]))

In [ ]:
# For para dar tratamiento a las líneas de compra que tuvieron tuvieron match con una misma línea de venta
wep_igual_uno = []
wep_muchas = []
wep_vacias = []

lista_ventas_merged = []

for i in range(len(compras_prod_varios_prov_odoo_after_merge1)):

    compra_df_for = compras_prod_varios_prov_odoo_after_merge1.iloc[i]

    mini_df = ventas_prod_varios_prov_after_merge1.loc[
                ~(ventas_prod_varios_prov_after_merge1['fact_line_id'].isin(lista_ventas_merged))
                & (ventas_prod_varios_prov_after_merge1['product_id'] == compra_df_for['product_id_pp'])
                & (ventas_prod_varios_prov_after_merge1['salesperson_name'] == compra_df_for['vendedora'])
                & (ventas_prod_varios_prov_after_merge1['quantity'] == compra_df_for['product_qty'])
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] >= compra_df_for['order_date'] - pd.Timedelta(days=15))
                & (ventas_prod_varios_prov_after_merge1['invoice_date'] <= compra_df_for['order_date'] + pd.Timedelta(days=3))
            ]

    if not mini_df.empty:
        lista_ventas_merged.append(mini_df['fact_line_id'].iloc[0])

    if len(mini_df) == 1:
        wep_igual_uno.append(compra_df_for['line_id'])

    if len(mini_df) > 1:
        wep_muchas.append(compra_df_for['line_id'])
    
    if mini_df.empty:
        wep_vacias.append(compra_df_for['line_id'])

print(len(wep_igual_uno), 'wep_igual_uno')
print(len(wep_muchas), 'wep_muchas')
print(len(wep_vacias), 'wep_vacias')
lista_ventas_merged

In [ ]:
a = compras_prod_varios_prov_odoo_after_merge1[~compras_prod_varios_prov_odoo_after_merge1['line_id'].isin(wep_igual_uno)].groupby('product_id_pp').agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']})
a['wep'] = a[('product_price', 'first')] == a[('product_price', 'mean')]
a.loc[a['wep'] == False]

In [ ]:
a = compras_prod_varios_prov_odoo_after_merge1.groupby('product_id_pp').agg({'product_name':['first'], 'line_id':['count'], 'product_price':['first', 'mean']})
a['wep'] = a[('product_price', 'first')] == a[('product_price', 'mean')]
a.loc[a['wep'] == False, ('line_id', 'count')].sum()
a.loc[a['wep'] == False]

In [ ]:
wep_igual_uno = []
wep_muchas = []
wep_vacias = []

for i in range(len(ventas_prod_varios_prov_after_merge1)):
    
    ventas = ventas_prod_varios_prov_after_merge1.iloc[i]

    mini_df = compras_prod_varios_prov_odoo_after_merge1.loc[
                (compras_prod_varios_prov_odoo_after_merge1['product_id_pp'] == ventas['product_id'])
                & (compras_prod_varios_prov_odoo_after_merge1['vendedora'] == ventas['salesperson_name'])
                & (compras_prod_varios_prov_odoo_after_merge1['product_qty'] == ventas['quantity'])
                & (compras_prod_varios_prov_odoo_after_merge1['order_date'] >= ventas['invoice_date'] - pd.Timedelta(days=5))
                & (compras_prod_varios_prov_odoo_after_merge1['order_date'] <= ventas['invoice_date'] + pd.Timedelta(days=5))
            ]

    if len(mini_df) == 1:
        wep_igual_uno.append(i)

    if len(mini_df) > 1:
        wep_muchas.append(i)
    
    if mini_df.empty:
        wep_vacias.append(i)

print(len(wep_igual_uno), 'wep_igual_uno')
print(len(wep_muchas), 'wep_muchas')
print(len(wep_vacias), 'wep_vacias')

Después de corregir el costo de la venta... las siguientes líneas ya son de utilidades

In [ ]:
# #Borrar esta línea, es provisional. Se tienen que corregir usando pd.NaN

# for i in range(len(ventas_año['price_subtotal'])):
#     ventas_año['price_subtotal'].iloc[i] = 0.01 if ventas_año['price_subtotal'].iloc[i] == 0 else ventas_año['price_subtotal'].iloc[i]

In [ ]:
# ventas_año['costo_date_dif'] = ((ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days).astype('Int64')
# ventas_año['costo_subtotal'] = ventas_año['quantity'] * ventas_año['costo_producto']
# ventas_año['utilidad_subtotal'] = ventas_año['price_subtotal'] - ventas_año['costo_subtotal']
# ventas_año['utilidad_%'] = ((ventas_año['price_subtotal'] / ventas_año['costo_subtotal']) - 1) * 100
# ventas_año['marg_util_%'] = (ventas_año['utilidad_subtotal'] / ventas_año['price_subtotal']) * 100


# cols_ventas = ['fact_doc_id', 'name', 'invoice_date', 'partner_id',
#        'partner_name', 'salesperson_id', 'salesperson_name', 'sale_team_description', 'business_model',
#        'product_id', 'product_name', 'quantity', 'price_subtotal',
#        'costo_subtotal', 'costo_order_date', 'costo_order_line_id',
#        'costo_date_dif', 'utilidad_subtotal', 'utilidad_%', 'marg_util_%', 'costo_producto']

# ventas = ventas_año[cols_ventas]